In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from operator import itemgetter
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import ChatPromptTemplate
from langchain_community.vectorstores import DocArrayInMemorySearch

In [2]:
MODEL = "mistral:7b"
model = Ollama(model = MODEL)
embedings = OllamaEmbeddings(model = MODEL)

C:\Users\DHRUV\AppData\Local\Temp\ipykernel_27864\3700191053.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model = MODEL)
C:\Users\DHRUV\AppData\Local\Temp\ipykernel_27864\3700191053.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedings = OllamaEmbeddings(model = MODEL)


In [3]:
prompt_template = """

Answer the question from the content given 
if you dont know the answer then get reference
from your training data

Content : {content}
Question : {question}


"""

prompt = ChatPromptTemplate.from_template(prompt_template)

In [4]:
loader = PyPDFLoader(file_path=r"C:\Users\DHRUV\Desktop\IProject\Papers and Refrence Codes\2303.11989v2.pdf")
pages = loader.load_and_split()
pages[:1]

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-09-12T02:06:26+00:00', 'author': '', 'keywords': '', 'moddate': '2023-09-12T02:06:26+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'C:\\Users\\DHRUV\\Desktop\\IProject\\Papers and Refrence Codes\\2303.11989v2.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='Text2Room: Extracting Textured 3D Meshes from 2D Text-to-Image Models\nLukas H¨ollein1∗ Ang Cao2∗ Andrew Owens2 Justin Johnson2 Matthias Nießner1\n1Technical University of Munich 2University of Michigan\n"a livingroomwithlots ofbookshelves, couches,and smalltables"\n… …\nIterativeGeneration\n"a livingroomwithlots ofbookshelves, couches,and smalltables"\n… …\nIterativeGeneration\n(a) 3D Mesh Generation from Text (b) Rendered Image + Mesh\nFigure 1. Textured 3D mesh generation from 

In [5]:
vector_store = DocArrayInMemorySearch.from_documents(pages,embedding=embedings)
retriver = vector_store.as_retriever()

c:\Users\DHRUV\anaconda3\envs\rag\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [6]:
retriver.invoke("What is this paper bout",k =2)

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-09-12T02:06:26+00:00', 'author': '', 'keywords': '', 'moddate': '2023-09-12T02:06:26+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'C:\\Users\\DHRUV\\Desktop\\IProject\\Papers and Refrence Codes\\2303.11989v2.pdf', 'total_pages': 18, 'page': 10, 'page_label': '11'}, page_content='on Computer Vision and Pattern Recognition (CVPR), pages\n10674–10685, 2021. 1, 2, 5, 13, 15\n[67] Olaf Ronneberger, Philipp Fischer, and Thomas Brox. U-\nnet: Convolutional networks for biomedical image segmen-\ntation. In Medical Image Computing and Computer-Assisted\nIntervention–MICCAI 2015: 18th International Conference,\nMunich, Germany, October 5-9, 2015, Proceedings, Part III\n18, pages 234–241. Springer, 2015. 2\n[68] Chitwan Saharia, William Chan, Saurabh Saxena, Lala\

In [7]:
chain = {"content" : itemgetter("question") | retriver, "question" : itemgetter("question")} | prompt | model

In [8]:
question = input("Enter your query : ")
chain.invoke({"question" : question})

' The PDF appears to be a research paper titled "Tune-a-video: One-shot tuning of image diffusion models for text-to-video generation" by authors Yuchao Gu, Wynne Hsu, Ying Shan, Xiaohu Qie, and Mike Zheng Shou. The paper discusses a method for generating realistic 3D scenes from text descriptions and synthesizing videos from these generated scenes.'